# TP Représentation symbolique du langage naturel et inférence

In [2]:
import nltk

## Sémantique compositionnelle

Dorénavant nous allons nous intéresser à la construction d'un énoncé logique correspondant à un arbre d'analyse.

Chaque non-terminal de la grammaire va remonter une caractéristique (*feature*) associée supposée représenter la sémantique du groupe représenté par ce non-terminal. Il serait agréable que chaque groupe identique puisse utiliser la même sémantique, mais la langue naturelle  est complexe et il faudra parfois différenceier selon l'usage (par exemple : sujet versus complément).

Nous allons construire la sémantique de l'énoncé global en combinant des éléments sémantiques atomiques.
Pour ce faire, nous allons nous aider de $\lambda$-expressions.

## Expressions logiques

NLTK permet de manipuler des expressions logiques et d'effectuer toutes les opérations nécessaires à la construction d'une démonstration, en logique propositionnelle ou en logique du premier ordre.

In [3]:
read_expr = nltk.sem.Expression.fromstring
e1 = read_expr('forall x.(student(x) -> person(x))')
e2 = read_expr('student(alice)')
e3 = read_expr('-person(alice)')
print(e1, e2, e3)

all x.(student(x) -> person(x)) student(alice) -person(alice)


Nous souhaitons exprimer que "quelqu'un est un étudiant".
Le formalisme logique nous permet d'énoncer que :

$\forall x student(x)$

ou que :

$\exists x student(x)$

Ce dont nous allons avoir besoin est un peu différent. Nous avons besoin d'exprimer par une fonction la qualité d'être étudiant. 
L'application de la fonction à un argument permettra d'instancier le terme. 
Les notions de fonction et d'application sont des attributs essentiels du lambda calcul.

## $\lambda$-expressions

Nous allons utiliser le $\lambda$-calcul (voir polycopié) pour dénoter des ensembles de la façon suivante :
$$\lambda x . [ \text{formule logique comprenant la variable } x ]$$

Considérons la phrase : "someone gives something to Alice", alors l'expression :
$$\lambda y . give(bob, alice, y)$$
définit la fonction caractéristique des objets $y$ que Bob donne à Alice.

De même :
$$\lambda x y. give(x, alice, y)$$
définit la fonction caractéristique des couples $(x, y)$ de donneurs et d'objets donnés à Alice.

Si $F$ est une formule logique, alors $\lambda x. F$ désignera l'ensemble défini au moyen de $F$ des $x$ qui rendent la formule $F$ vraie.

**Syntaxe :** Soit $F$ une formule logique et $x_1, \ldots, x_n$ $n$ variables, alors l'expression $\lambda x_1 \ldots x_n . F$ est une $\lambda$-expression dont $F$ est le corps.

**Sémantique :** Les valeurs de la $\lambda$-expression $\lambda x_1 \ldots x_n . F$ sont les valeurs du prédicat n-aire obtenu en attribuant des instances aux paramètres formels.

### Application - Substitution - Réduction

En $\lambda$-calcul, tout est fonction.

**Syntaxe :$$ 
- les variables : $x$, $y$... sont des $\lambda$-termes ;
- les applications : $u v$ est un $\lambda$-terme si $u$ et $v$ sont des $\lambda$-termes ;
- les abstractions : $\lambda x.v$ est un $\lambda$-terme si $x$ est une variable et $v$ un $\lambda$-terme.

Exemple d'application : $(\lambda x. student(x))(alice)$

Autre exemple d'application : $(\lambda x y. x + y)(\lambda z . z * 2)$

#### Substitution

On définit la substition dans les $\lambda$-termes de façon similaire à la substitution dans les termes logiques, en prenant garde aux captures possibles de variables. Exemples :

- $(\lambda x. x y)[y / a] = (\lambda x. x a)$
- $(\lambda x. x y)[y / x] = (\lambda z. z x)$ (et non $(\lambda x. x x)$, qui est totalement différent, à cause de la capture)

L'$\alpha$-conversion identifie $\lambda y. v$ et $\lambda z. v[y / z]$. C'est une relation d'équivalence entre $\lambda$-termes.

#### Réduction

On définit la bêta-contraction (ou $\beta$-contraction) de $(\lambda x. u) v$ comme $u[x / v]$.

Exemple : $(\lambda x. x y) a$ donne $(x y)[x / a] = ay$.

La réduction consiste à appliquer la $\beta$-contraction autant de fois que possible pour mettre le terme en forme normale réduite (si possible).

**Il n'est pas question ici d'expliquer le $\lambda$-calcul dans toute sa généralité mais seulement d'en utiliser une version non-typée à des fins très pratiques de combinaison de formules logiques.**

### NLTK et $\lambda$-expressions

NLTK permet de manipuler des $\lambda$-expressions (voir polycopié).
Ces expressions représentent des fonctions et le $\lambda$-calcul consiste à combiner des $\lambda$-expressions puis à les réduire.

Ici, la syntaxe des $\lambda$-expressions se combine avec celle des expressions logiques pour les rendre paramétriques et permettre de les combiner.

In [4]:
l1 = read_expr(r'(\x.student(x))(alice)')
l1.simplify()

<ApplicationExpression student(alice)>

En revanche, avec cette syntaxe, on ne peut pas espérer obtenir un résultat pour :

In [5]:
try:
    l2 = read_expr(r'(\x.x(alice))(student)')
    print(l2.simplify())
except nltk.LogicalExpressionException as e:
    print(f'ValueError {e}')

ValueError 'x' is an illegal predicate name.  Individual variables may not be used as predicates.
(\x.x(alice))(student)
    ^


Le $\lambda$-calcul permet ce type de substitution, et pour y accéder avec NLTK, il faut utiliser des variables en lettres capitales qui peuvent se substituer à des prédicats :

In [6]:
try:
    l2 = read_expr(r'(\X.X(alice))(student)')
    print(l2.simplify())
except nltk.LogicalExpressionException as e:
    print(f'ValueError {e}')

student(alice)


Nous allons donc pouvoir combiner plusieurs expressions logiques fonctionnelles pour construire des formules élaborées représentant la sémantique d'un énoncé donné.

Soient les expressions suivantes :

In [7]:
subcategory = read_expr(r'\P Q.(all x. P(x) -> Q(x))')
student = read_expr(r'\x.student(x)')
person = read_expr(r'\x.person(x)')

On peut appliquer la règle subcategory aux catégories student et person :

In [8]:
print(subcategory(student)(person).simplify())

(all x.student(x) -> person(x))


## Sémantique de la langue naturelle

Nous allons associer à chaque symbole terminal ou non une $\lambda$-expression que l'on appellera sa sémantique. Ces expressions seront composées par les règles pour remonter une expression au noeud de départ. Cette expression sera la traduction en langage logique de notre énoncé en langage naturel. Nous allons utiliser les *features* pour remonter la sémantique de la phrase analysée.

### Note linguistique

Le traitement des verbes transitifs (comme donner) ou intransitifs (comme aller) est différent de celui des verbes que l'on qualifie de *verbes d'état* en français (comme être). Il ne faut donc pas être étonné d'avoir des règles spécifiques pour être quand il est utilisé pour catégoriser. On a une illustration sur les deux exemple suivants.


In [9]:
def translate(grammar, sents, trace=0):
    '''
    Parses the sentences given.
    Create a feature chart parser,
    split the sentences into tokens
    and parse.
    Returns the SEM feature at the start symbol.
    Beware: do not forget to state the start symbol with % start.
    '''
    parser = nltk.FeatureChartParser(grammar, trace=trace)
    for sent in sents:
        tokens = sent.split()
        trees = parser.parse(tokens)
        for tree in trees:
            print(f'{sent}\t->\t{tree.label()["SEM"]}')

In [10]:
grammar = nltk.grammar.FeatureGrammar.fromstring('''
% start S
S[SEM=<?sv(?sn)>] -> GN[NUM=?n,SEM=?sn] GV[NUM=?n,SEM=?sv]
GV[NUM=?n,SEM=<(?sv)(?scod)(\Y.Y)>] -> VTrans[NUM=?n,SEM=?sv] GN[SEM=?scod] 
GV[NUM=?n,SEM=<(?sv)(?scod)(?scoi)>] -> VTrans[NUM=?n,SEM=?sv] GN[SEM=?scod] Prep GN[SEM=?scoi]
GN[NUM=sg,SEM=?sn] -> N[NUM=sg,SEM=?sn] | Det[NUM=sg] N[NUM=sg,SEM=?sn]
GN[NUM=pl,SEM=?sn] -> N[NUM=pl,SEM=?sn]
Det[NUM=sg] -> 'the'
Det[NUM=sg] -> 'a'
VTrans[NUM=sg,SEM=<\X Y Z.Z(\\x.X(\z.Y(\y.give(x,y,z))))>] -> 'gives'
N[NUM=sg,SEM=<\P.P(alice)>] -> 'Alice'
N[NUM=sg,SEM=<\P.P(bob)>] -> 'Bob'
N[NUM=sg,SEM=<\P.P(book)>] -> 'book' 
N[NUM=pl,SEM=<\P.P(book)>] -> 'books' 
N[NUM=sg,SEM=<\P.P(candy)>] -> 'candy' 
N[NUM=pl,SEM=<\P.P(candies)>] -> 'candies' 
Prep -> 'to'
''')

In [11]:
sents = ['Bob gives a book to Alice', 
         'Bob  gives candies']
translate(grammar, sents, trace=0)

Bob gives a book to Alice	->	give(bob,alice,book)
Bob  gives candies	->	\y.give(bob,y,candies)


In [12]:
grammar = nltk.grammar.FeatureGrammar.fromstring('''
% start S
S[SEM=<?sv(?sn)>] -> GN[NUM=?n,SEM=?sn] GV[NUM=?n,SEM=?sv]
GV[NUM=?n,SEM=<(?sv)(?sn)>] -> VState[NUM=?n,SEM=?sv] GN[SEM=?sn] 
GN[NUM=sg,SEM=?sn] -> N[NUM=sg,SEM=?sn] | Det[NUM=sg] N[NUM=sg,SEM=?sn]
GN[NUM=pl,SEM=?sn] -> N[NUM=pl,SEM=?sn]
Det[NUM=sg] -> 'the'
Det[NUM=sg] -> 'a'
VState[NUM=sg,SEM=<\X Y.X(Y)>] -> 'is'
N[NUM=sg,SEM=<\P.P(alice)>] -> 'Alice'
N[NUM=sg,SEM=<\P.P(bob)>] -> 'Bob'
N[NUM=sg,SEM=<\P.P(student)>] -> 'student' 
''')

In [13]:
sents = ['Bob is a student']
translate(grammar, sents, trace=0)

Bob is a student	->	student(bob)


## Travail

Augmenter la grammaire ci-dessus pour analyser la phrase :

Alice and Bob are students -> student(alice) and student(bob)

In [14]:
grammar = nltk.grammar.FeatureGrammar.fromstring('''
% start S
S[SEM=<?sv(?sn)>] -> GN[NUM=?n,SEM=?sn]  GV[NUM=?n,SEM=?sv]
GV[NUM=?n,SEM=<(?sv)(?sn)>] -> VState[NUM=?n,SEM=?sv] GN[SEM=?sn] 
GN[NUM=sg,SEM=?sn] -> N[NUM=sg,SEM=?sn] | Det[NUM=sg] N[NUM=sg,SEM=?sn] 
GN[NUM=pl,SEM=?sn] -> N[NUM=pl,SEM=?sn] 
GN[NUM=pl,SEM=<?snconj(?sn1)(?sn2)>] -> N[SEM=?sn1] Conj[SEM=?snconj] N[SEM=?sn2] 
Det[NUM=sg] -> 'the'
Det[NUM=sg] -> 'a'
VState[NUM=sg,SEM=<\X Y.X(Y)>] -> 'is'
VState[NUM=pl,SEM=<\X Y.X(Y)>] -> 'are'
N[NUM=sg,SEM=<\P.P(alice)>] -> 'Alice'
N[NUM=sg,SEM=<\P.P(bob)>] -> 'Bob'
N[NUM=sg,SEM=<\P.P(student)>] -> 'student' 
N[NUM=pl,SEM=<\P.P(student)>] -> 'students' 
Conj[SEM=<\Q R P.(Q(P) & R(P))>] -> 'and'
''')

In [15]:
sents = ['Alice and Bob are students']
translate(grammar, sents, trace=1)

|.Al.an.Bo.ar.st.|
|[--]  .  .  .  .| [0:1] 'Alice'
|.  [--]  .  .  .| [1:2] 'and'
|.  .  [--]  .  .| [2:3] 'Bob'
|.  .  .  [--]  .| [3:4] 'are'
|.  .  .  .  [--]| [4:5] 'students'
|[--]  .  .  .  .| [0:1] N[NUM='sg', SEM=<\P.P(alice)>] -> 'Alice' *
|[--]  .  .  .  .| [0:1] GN[NUM='sg', SEM=<\P.P(alice)>] -> N[NUM='sg', SEM=<\P.P(alice)>] *
|[-->  .  .  .  .| [0:1] GN[NUM='pl', SEM=<?snconj(?sn1,?sn2)>] -> N[SEM=?sn1] * Conj[SEM=?snconj] N[SEM=?sn2] {?sn1: <LambdaExpression \P.P(alice)>}
|[-->  .  .  .  .| [0:1] S[SEM=<?sv(?sn)>] -> GN[NUM=?n, SEM=?sn] * GV[NUM=?n, SEM=?sv] {?n: 'sg', ?sn: <LambdaExpression \P.P(alice)>}
|.  [--]  .  .  .| [1:2] Conj[SEM=<\Q R P.(Q(P) & R(P))>] -> 'and' *
|[----->  .  .  .| [0:2] GN[NUM='pl', SEM=<?snconj(?sn1,?sn2)>] -> N[SEM=?sn1] Conj[SEM=?snconj] * N[SEM=?sn2] {?sn1: <LambdaExpression \P.P(alice)>, ?snconj: <LambdaExpression \Q R P.(Q(P) & R(P))>}
|.  .  [--]  .  .| [2:3] N[NUM='sg', SEM=<\P.P(bob)>] -> 'Bob' *
|.  .  [--]  .  .| [2:3] GN[NUM='sg',

Ajouter la négation :

Alice is not a student -> -student(alice)


In [27]:
grammar = nltk.grammar.FeatureGrammar.fromstring('''
% start S
S[SEM=<?sv(?sn)>] -> GN[NUM=?n,SEM=?sn] GV[NUM=?n,SEM=?sv]
GV[NUM=?n,SEM=<(?sv)(?neg)(?sn)>] -> VState[NUM=?n,SEM=?sv] GN[SEM=?sn] | VState[NUM=?n,SEM=?sv] ADV[SEM=?neg] GN[SEM=?sn]
GN[NUM=sg,SEM=?sn] -> N[NUM=sg,SEM=?sn] | Det[NUM=sg] N[NUM=sg,SEM=?sn]
GN[NUM=pl,SEM=?sn] -> N[NUM=pl,SEM=?sn]
GN[NUM=pl,SEM=<?snconj(?sn1)(?sn2)>] -> N[SEM=?sn1] Conj[SEM=?snconj] N[SEM=?sn2] 
Det[NUM=sg] -> 'the'
Det[NUM=sg] -> 'a'
Conj[SEM=<\Q R P . (Q(P) & R(P))>] -> 'and'
ADV[SEM=<\P x. (-P(x))>] -> 'not'                                                                                                  
VState[NUM=sg,SEM=<\X Y.X(Y)>] -> 'is'
VState[NUM=pl,SEM=<\X Y.X(Y)>] -> 'are'
N[NUM=sg,SEM=<\P.P(alice)>] -> 'Alice'
N[NUM=sg,SEM=<\P.P(bob)>] -> 'Bob'
N[NUM=sg,SEM=<\P.P(student)>] -> 'student'                                                  
N[NUM=pl,SEM=<\P.P(student)>] -> 'students'
''')

In [28]:
sents = ['Alice is not a student']
translate(grammar, sents, trace=1)

|.Al.is.no.a .st.|
|[--]  .  .  .  .| [0:1] 'Alice'
|.  [--]  .  .  .| [1:2] 'is'
|.  .  [--]  .  .| [2:3] 'not'
|.  .  .  [--]  .| [3:4] 'a'
|.  .  .  .  [--]| [4:5] 'student'
|[--]  .  .  .  .| [0:1] N[NUM='sg', SEM=<\P.P(alice)>] -> 'Alice' *
|[--]  .  .  .  .| [0:1] GN[NUM='sg', SEM=<\P.P(alice)>] -> N[NUM='sg', SEM=<\P.P(alice)>] *
|[-->  .  .  .  .| [0:1] GN[NUM='pl', SEM=<?snconj(?sn1,?sn2)>] -> N[SEM=?sn1] * Conj[SEM=?snconj] N[SEM=?sn2] {?sn1: <LambdaExpression \P.P(alice)>}
|[-->  .  .  .  .| [0:1] S[SEM=<?sv(?sn)>] -> GN[NUM=?n, SEM=?sn] * GV[NUM=?n, SEM=?sv] {?n: 'sg', ?sn: <LambdaExpression \P.P(alice)>}
|.  [--]  .  .  .| [1:2] VState[NUM='sg', SEM=<\X Y.X(Y)>] -> 'is' *
|.  [-->  .  .  .| [1:2] GV[NUM=?n, SEM=<?sv(?neg,?sn)>] -> VState[NUM=?n, SEM=?sv] * GN[SEM=?sn] {?n: 'sg', ?sv: <LambdaExpression \X Y.X(Y)>}
|.  [-->  .  .  .| [1:2] GV[NUM=?n, SEM=<?sv(?neg,?sn)>] -> VState[NUM=?n, SEM=?sv] * ADV[SEM=?neg] GN[SEM=?sn] {?n: 'sg', ?sv: <LambdaExpression \X Y.X(Y)>}
|.  